In [1]:
from configparser import ConfigParser
import pandas as pd
from utils import *

In [2]:
#Config file
config_dir = "config/config.ini"
config = ConfigParser()
config.read(config_dir)

#API config variables
api_key = config["openaq_api_credentials"]["api_key"]

#Create Redshift Conn based on config file
conn = connect_to_db(config, "redshift_credentials")


2023-11-12 13:48:01,784 - INFO - Conexión a la base de datos establecida exitosamente.


In [3]:
#set OpenQA API base url and headers
base_url = "https://api.openaq.org/v2"

headers = {
    "X-API-Key" : api_key,
    "accept": "application/json",
    "content-type": "application/json"
}

In [57]:
#Get locations from OpenAQ API
endpoint = "locations"
json_api=get_data(base_url, endpoint, headers)
df_api = pd.json_normalize(json_api)

df_api.drop(['parameters','measurements','manufacturers','bounds'],axis=1,inplace=True )

load_to_sql(
    df= df_api,
    table_name= "stg_locations",
    engine= conn,
    if_exists= 'replace',
    )


2023-11-11 18:36:37,828 - INFO - Obteniendo datos de https://api.openaq.org/v2/locations...
2023-11-11 18:36:37,922 - INFO - Cargando datos en la base de datos...
/Users/nico/Library/Python/3.9/lib/python/site-packages/sqlalchemy/engine/default.py:736: UserWarning: DB-API extension cursor.connection used
  cursor.execute(statement, parameters)
2023-11-11 18:36:37,924 - ERROR - Error al cargar los datos en la base de datos: [Errno 32] Broken pipe


In [51]:
df_api['manufacturers']

0     [{'modelName': 'Government Monitor', 'manufact...
1     [{'modelName': 'Government Monitor', 'manufact...
2     [{'modelName': 'Government Monitor', 'manufact...
3     [{'modelName': 'Government Monitor', 'manufact...
4     [{'modelName': 'Government Monitor', 'manufact...
                            ...                        
95    [{'modelName': 'PurpleAir Sensor', 'manufactur...
96    [{'modelName': 'PurpleAir Sensor', 'manufactur...
97    [{'modelName': 'PurpleAir Sensor', 'manufactur...
98    [{'modelName': 'PurpleAir Sensor', 'manufactur...
99    [{'modelName': 'PurpleAir Sensor', 'manufactur...
Name: manufacturers, Length: 100, dtype: object

In [58]:
#Get parameters from OpenAQ API 
endpoint = "parameters"
json_api=get_data(base_url, endpoint, headers)
df_api = pd.json_normalize(json_api)

df_api

2023-11-12 12:09:25,383 - INFO - Obteniendo datos de https://api.openaq.org/v2/parameters...


,id,name,displayName,description,preferredUnit
0,1,pm10,PM10,Particulate matter less than 10 micrometers in...,µg/m³
1,2,pm25,PM2.5,Particulate matter less than 2.5 micrometers i...,µg/m³
2,3,o3,O₃ mass,Ozone mass concentration,µg/m³
3,4,co,CO mass,Carbon Monoxide mass concentration,µg/m³
4,5,no2,NO₂ mass,Nitrogen Dioxide mass concentration,µg/m³
...,...,...,...,...,...
59,19856,ca,Ca,Calcium,ppb
60,19857,fe,Fe,Iron,ppb
61,19858,ni,Ni,Nickle,ppb
62,19859,v,V,Vanadium,ppb


In [59]:
#Get measures from OpenAQ API
endpoint = "measurements"
start_date = datetime.utcnow() - timedelta(hours=24)  #datos de las ultimas 24hs..
params = {
    "date_from" : start_date
}

json_api=get_data(base_url, endpoint, headers)
df_api = pd.json_normalize(json_api)

df_api


2023-11-12 12:45:16,947 - INFO - Obteniendo datos de https://api.openaq.org/v2/measurements...


,locationId,location,parameter,value,unit,country,city,isMobile,isAnalysis,entity,sensorType,date.utc,date.local,coordinates.latitude,coordinates.longitude
0,68082,Alta Drive,um005,0.998276,particles/cm³,US,None,False,None,Community Organization,low-cost sensor,2023-11-12T10:00:00+00:00,2023-11-12T03:00:00-07:00,40.59164,-111.80380
1,68082,Alta Drive,um003,3.572759,particles/cm³,US,None,False,None,Community Organization,low-cost sensor,2023-11-12T10:00:00+00:00,2023-11-12T03:00:00-07:00,40.59164,-111.80380
2,68082,Alta Drive,pm1,1.068966,µg/m³,US,None,False,None,Community Organization,low-cost sensor,2023-11-12T10:00:00+00:00,2023-11-12T03:00:00-07:00,40.59164,-111.80380
3,68082,Alta Drive,temperature,70.000000,f,US,None,False,None,Community Organization,low-cost sensor,2023-11-12T10:00:00+00:00,2023-11-12T03:00:00-07:00,40.59164,-111.80380
4,68082,Alta Drive,pm10,2.293103,µg/m³,US,None,False,None,Community Organization,low-cost sensor,2023-11-12T10:00:00+00:00,2023-11-12T03:00:00-07:00,40.59164,-111.80380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1823076,801 52nd street,voc,65.023333,iaq,US,None,False,None,Person,low-cost sensor,2023-11-12T10:00:00+00:00,2023-11-12T02:00:00-08:00,38.56818,-121.43982
96,1823076,801 52nd street,um003,60.711667,particles/cm³,US,None,False,None,Person,low-cost sensor,2023-11-12T10:00:00+00:00,2023-11-12T02:00:00-08:00,38.56818,-121.43982
97,1823076,801 52nd street,um005,17.662000,particles/cm³,US,None,False,None,Person,low-cost sensor,2023-11-12T10:00:00+00:00,2023-11-12T02:00:00-08:00,38.56818,-121.43982
98,1823076,801 52nd street,pm10,53.890000,µg/m³,US,None,False,None,Person,low-cost sensor,2023-11-12T10:00:00+00:00,2023-11-12T02:00:00-08:00,38.56818,-121.43982


In [4]:
#Get locations from OpenAQ API
endpoint = "locations"
json_api=get_data(base_url, endpoint, headers)
df_api = pd.json_normalize(json_api)
df_api

2023-11-10 19:12:15,140 - INFO - Obteniendo datos de https://api.openaq.org/v2/locations...


,id,city,name,entity,country,sources,isMobile,isAnalysis,parameters,sensorType,lastUpdated,firstUpdated,measurements,bounds,manufacturers,coordinates.latitude,coordinates.longitude
0,5490,Tasmania Region,Perth,None,AU,None,False,None,"[{'id': 1, 'unit': 'µg/m³', 'count': 45000, 'a...",None,2023-11-11T12:04:22+00:00,2017-07-13T14:24:26+00:00,89995,"[147.172, -41.578, 147.172, -41.578]","[{'modelName': 'Government Monitor', 'manufact...",-41.578000,147.17200
1,5485,Tasmania Region,Deloraine,None,AU,None,False,None,"[{'id': 1, 'unit': 'µg/m³', 'count': 44903, 'a...",None,2023-11-11T12:04:16+00:00,2017-07-13T14:24:20+00:00,89804,"[146.658, -41.523, 146.658, -41.523]","[{'modelName': 'Government Monitor', 'manufact...",-41.523000,146.65800
2,233316,Tasmania Region,Latrobe,None,AU,None,False,None,"[{'id': 1, 'unit': 'µg/m³', 'count': 37091, 'a...",None,2023-11-11T12:04:11+00:00,2021-08-23T16:14:15+00:00,74182,"[146.42, -41.238, 146.42, -41.238]","[{'modelName': 'Government Monitor', 'manufact...",-41.238000,146.42000
3,5482,Tasmania Region,Emu River,None,AU,None,False,None,"[{'id': 1, 'unit': 'µg/m³', 'count': 2163, 'av...",None,2023-11-11T12:04:07+00:00,2022-10-03T18:14:07+00:00,4326,"[145.883, -41.108, 145.883, -41.108]","[{'modelName': 'Government Monitor', 'manufact...",-41.108000,145.88300
4,5483,Tasmania Region,Westbury,None,AU,None,False,None,"[{'id': 2, 'unit': 'µg/m³', 'count': 45256, 'a...",None,2023-11-11T12:04:07+00:00,2017-07-13T14:24:18+00:00,90511,"[146.835, -41.53, 146.835, -41.53]","[{'modelName': 'Government Monitor', 'manufact...",-41.530000,146.83500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,64994,None,P13 Air,None,US,None,False,None,"[{'id': 135, 'unit': 'particles/cm³', 'count':...",None,2023-11-10T17:54:01+00:00,2021-01-14T01:19:23+00:00,2701890,"[-118.40046, 33.765705, -118.40046, 33.765705]","[{'modelName': 'PurpleAir Sensor', 'manufactur...",33.765705,-118.40046
96,66236,None,Alta Heights,None,US,None,False,None,"[{'id': 19, 'unit': 'µg/m³', 'count': 453429, ...",None,2023-11-10T17:54:01+00:00,2021-01-20T00:00:11+00:00,3156612,"[-122.26393, 38.305733, -122.26393, 38.305733]","[{'modelName': 'PurpleAir Sensor', 'manufactur...",38.305733,-122.26393
97,66454,None,PET Rostraver 1,None,US,None,False,None,"[{'id': 134, 'unit': '%', 'count': 71947, 'ave...",None,2023-11-10T17:54:01+00:00,2021-01-19T23:59:52+00:00,3177480,"[-79.79551, 40.214046, -79.79551, 40.214046]","[{'modelName': 'PurpleAir Sensor', 'manufactur...",40.214046,-79.79551
98,66583,None,Mayhew's Landing,None,US,None,False,None,"[{'id': 126, 'unit': 'particles/cm³', 'count':...",None,2023-11-10T17:54:01+00:00,2021-01-19T23:59:53+00:00,3096870,"[-122.06041, 37.5349, -122.06041, 37.5349]","[{'modelName': 'PurpleAir Sensor', 'manufactur...",37.534900,-122.06041


In [4]:
#Get locations from OpenAQ API
endpoint = "locations"
json_api=get_data(base_url, endpoint, headers)
df_api = pd.json_normalize(json_api)

df_api.drop(['parameters','measurements','manufacturers','bounds'],axis=1,inplace=True )

column_mapping = {
    "coordinates.latitude": "coordinates_latitude",
    "coordinates.longitude": "coordinates_longitude"
}

load_to_sql(
    df= df_api,
    table_name= "stg_locations",
    engine= conn,
    if_exists= 'append',
    index_label=column_mapping.keys()
    )


2023-11-12 13:48:13,846 - INFO - Obteniendo datos de https://api.openaq.org/v2/locations...
2023-11-12 13:48:13,927 - INFO - Cargando datos en la tabla stg_locations...
/Users/nico/Library/Python/3.9/lib/python/site-packages/sqlalchemy/engine/default.py:736: UserWarning: DB-API extension cursor.connection used
  cursor.execute(statement, parameters)
2023-11-12 13:48:16,452 - ERROR - Error al cargar los datos en la base de datos: (redshift_connector.error.ProgrammingError) {'S': 'ERROR', 'C': '42703', 'M': 'column "coordinates.latitude" of relation "stg_locations" does not exist', 'F': '../src/pg/src/backend/parser/parse_relation.c', 'L': '4764', 'R': 'attnameAttNum'}
[SQL: INSERT INTO stg_locations (id, city, name, entity, country, sources, "isMobile", "isAnalysis", "sensorType", "lastUpdated", "firstUpdated", "coordinates.latitude", "coordinates.longitude") VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s), (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s), (%s, %s, %s, %

In [8]:
from configparser import ConfigParser
import pandas as pd
from utils import *
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

#Config file
config_dir = "config/config.ini"
config = ConfigParser()
config.read(config_dir)
logging.info("Archivo de configuracion leido correctamente.")

#API config variables
api_key = config["openaq_api_credentials"]["api_key"]

#Create Redshift Conn based on config file
engine = connect_to_db(config, "redshift_credentials")

#set OpenQA API base url and headers
base_url = "https://api.openaq.org/v2"

headers = {
    "X-API-Key" : api_key,
    "accept": "application/json",
    "content-type": "application/json"
}

#Get locations from OpenAQ API
endpoint = "locations"
json_api=get_data(base_url, endpoint, headers)
df_api = pd.json_normalize(data=json_api, sep='_')
df_api.drop(['parameters','measurements','manufacturers','bounds'],axis=1,inplace=True )

print(df_api)

with engine.connect() as conn, conn.begin():
    #conn.execute("TRUNCATE TABLE stg_locations")
    
    load_to_sql(
        df= df_api,
        table_name= "stg_locations",
        engine= engine,
        if_exists= 'append',
        )
    query="call pETL_load_dim_location()"
    conn.execute(query)
    logging.info(f"Se ejecuto correctamente la query: {query}")
    conn.close()

2023-11-12 17:51:30,575 - INFO - Archivo de configuracion leido correctamente.
2023-11-12 17:51:30,577 - INFO - Conexión a la base de datos establecida exitosamente.
2023-11-12 17:51:30,578 - INFO - Obteniendo datos de https://api.openaq.org/v2/locations...


        id             city                                   name entity  \
0     5481  Tasmania Region                               Longford   None   
1     5483  Tasmania Region                               Westbury   None   
2     5488  Tasmania Region                               Smithton   None   
3     5487  Tasmania Region                                Wynyard   None   
4     5491  Tasmania Region                        West Ulverstone   None   
..     ...              ...                                    ...    ...   
95  220444             None  Mid-Columbia Libraries Prosser Branch   None   
96  220595             None                          Pinecrest Air   None   
97  223915             None  1509 Cortez Ave, Burlingame, CA 94010   None   
98  224750             None                  TCES Back Left Sensor   None   
99  227135             None                       Smalley backyard   None   

   country sources  isMobile isAnalysis sensorType                lastUpdat

2023-11-12 17:51:32,616 - INFO - Cargando datos en la tabla stg_locations...
2023-11-12 17:51:34,697 - INFO - Datos cargados exitosamente en la tabla stg_locations.
2023-11-12 17:51:35,539 - INFO - Se ejecuto correctamente la query: call pETL_load_dim_location()


In [12]:
with engine.connect() as conn, conn.begin():
    conn.execute("TRUNCATE TABLE stg_locations;")
    
    load_to_sql(
        df= df_api,
        table_name= "stg_locations",
        engine=engine,
        if_exists= 'append',
        )
    query="call nicolasmvinciguerra_coderhouse.petl_load_dim_location();"
    conn.execute(query)
    logging.info(f"Se ejecuto correctamente la query: {query}")
    conn.close()

2023-11-12 18:00:31,354 - INFO - Cargando datos en la tabla stg_locations...
2023-11-12 18:00:32,121 - INFO - Datos cargados exitosamente en la tabla stg_locations.
2023-11-12 18:00:32,595 - INFO - Se ejecuto correctamente la query: call nicolasmvinciguerra_coderhouse.petl_load_dim_location();
